In [1]:
import os
from tool.dba import *

ModuleNotFoundError: No module named 'tool'

In [12]:
# date_range = pd.date_range(start='2021-01-01', end=datetime.datetime.now())
# df_date = pd.DataFrame(date_range, columns=["DS"])
# df_date["YEAR"] = df_date["DS"].dt.year
# df_date["MONTH"] = df_date["DS"].dt.month
# try:
#     os.makedirs("/home/zyuser/predict/data/配送数据")
#     os.makedirs("/home/zyuser/predict/data/退货数据")
# except FileExistsError:
#     logger.info(f"Path exists: /home/zyuser/predict/data/退货数据 ")
# for (year, month), g_data in df_date.groupby(["YEAR", "MONTH"], as_index=False):
#     print(year, month)
#     start_date, end_date = g_data["DS"].min(), g_data["DS"].max()
#     sql_th = f"""
#     SELECT ORGCODE STORE_ID, 
#             PLUCODE PRODUCT_ID, 
#             DZ_DATE DS,
#             PLUTOTAL TH 
#     FROM TZYPLUVIEW_TH
#     WHERE DZ_DATE >= TO_DATE('{start_date}','yyyy-mm-dd hh24:mi:ss') AND DZ_DATE <= TO_DATE('{end_date}','yyyy-mm-dd hh24:mi:ss') and PLUTYPE = 'GQTH'
#     """
#     df_th = oracle97.read(sql_th)
#     df_th.to_feather(os.path.join("/home/zyuser/predict/data/退货数据", "%04d%02d.feather" % (year, month)))
#     sql_ps = f"""
#     SELECT ORGCODE STORE_ID, 
#             PLUCODE PRODUCT_ID, 
#             RPTDATE DS,
#             PLUTOTAL PS 
#     FROM TZYPLUPRO_PS
#     WHERE RPTDATE >= TO_DATE('{start_date}','yyyy-mm-dd hh24:mi:ss') AND RPTDATE <= TO_DATE('{end_date}','yyyy-mm-dd hh24:mi:ss')
#     """
#     df_ps = oracle97.read(sql_ps)
#     df_ps.to_feather(os.path.join("/home/zyuser/predict/data/配送数据", "%04d%02d.feather" % (year, month)))

date_range = pd.date_range(start='2022-01-01', end='2022-06-30')
df_date = pd.DataFrame(date_range, columns=["DS"])
df_date["YEAR"] = df_date["DS"].dt.year
df_date["MONTH"] = df_date["DS"].dt.month
try:
    os.makedirs("/home/zyuser/predict/data/配送数据")
    os.makedirs("/home/zyuser/predict/data/退货数据")
except FileExistsError:
    logger.info(f"Path exists: /home/zyuser/predict/data/退货数据 ")
for (year, month), g_data in df_date.groupby(["YEAR", "MONTH"], as_index=False):
    print(year, month)
    start_date, end_date = g_data["DS"].min(), g_data["DS"].max()
    sql_th = f"""
    SELECT DD.DZ_DATE DS,
       DD.PRODUCT_STORE_ID STORE_ID,
       DI.PRODUCT_ID,
       NVL(DI.AMOUNT3, 0) TH
    FROM ZDEV_TH_DOC DD
    INNER JOIN ZDEV_TH_ITEM DI ON DD.DOC_ID = DI.DOC_ID
    WHERE DZ_DATE >= TO_DATE('{start_date}','yyyy-mm-dd hh24:mi:ss') AND DZ_DATE <= TO_DATE('{end_date}','yyyy-mm-dd hh24:mi:ss') and DD.FAULT_TYPE_ID = '10001'
    """
    df_th = oracle97.read(sql_th)
    df_th.to_feather(os.path.join("/home/zyuser/predict/data/退货数据", "%04d%02d.feather" % (year, month)))
    sql_ps = f"""
    SELECT
        DD.POSTING_DATE AS DS,
        DD.PRODUCT_STORE_ID_TO AS STORE_ID,
        DI.PRODUCT_ID AS PRODUCT_ID,
        NVL(DI.AMOUNT,0) PS
        FROM ZDEV_PS_DOC DD
        INNER JOIN ZDEV_PS_ITEM DI ON DD.DOC_ID=DI.DOC_ID
        WHERE DD.MOVEMENT_TYPE_ID IN ('PS','DR','DZ')
    AND POSTING_DATE >= TO_DATE('{start_date}','yyyy-mm-dd hh24:mi:ss') AND POSTING_DATE <= TO_DATE('{end_date}','yyyy-mm-dd hh24:mi:ss')
    """
    df_ps = oracle97.read(sql_ps)
    df_ps.to_feather(os.path.join("/home/zyuser/predict/data/配送数据", "%04d%02d.feather" % (year, month)))

2022 1
2022 2
2022 3
2022 4
2022 5
2022 6


In [13]:
path = "/home/zyuser/predict/data/配送数据"
df_ps = pd.concat([pd.read_feather(os.path.join(path, file)) for file in os.listdir(path)])
df_ps = df_ps[df_ps["PS"] > 0]
path = "/home/zyuser/predict/data/退货数据"
df_th = pd.concat([pd.read_feather(os.path.join(path, file)) for file in os.listdir(path)])
df_th = df_th[df_th["TH"] > 0]

In [108]:
plu = oracle97.read("SELECT PRODUCT_ID FROM PLUVIEW WHERE PRODUCT_CATEGORY_NAME='散装食品' and UOM='千克'")
store = oracle97.read("SELECT PRODUCT_STORE_ID FROM ORGVIEW WHERE STORE_BRAND IN ('紫燕百味鸡', '钟记油烫鸭')")

In [ ]:
df = pd.concat([df_ps, df_th])
df = df[df.STORE_ID.isin(store.PRODUCT_STORE_ID)]
df = df[df.PRODUCT_ID.isin(plu.PRODUCT_ID)]
df["YEAR"] = df["DS"].dt.year
df["MONTH"] = df["DS"].dt.month
# df = df[df["MONTH"] <= 6]
df = pd.pivot_table(df, columns="YEAR", values=["PS", "TH"], aggfunc="sum", index=["STORE_ID", "PRODUCT_ID"], fill_value=0).reset_index()
df

In [ ]:
df.columns = [var[0] + str(var[1]) for var in df.columns]
df.columns

In [ ]:
df["RATE2021"] = df["TH2022"].cumsum() / df["PS2022"].cumsum()
df = df.sort_values(by="TH2022", ascending=True).reset_index(drop=True)
df["RATE2021"] = df["TH2021"].cumsum() / df["PS2021"].cumsum()
df["RATE2022"] = df["TH2022"].cumsum() / df["PS2022"].cumsum()

In [ ]:
df

In [ ]:
df.to_csv("退货数据.csv")